In [1]:
import pandas as pd
import re
import numpy as np
import itertools
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from razdel import tokenize

import pymorphy2  

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aszhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
np.random.seed(42)

In [3]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [4]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [5]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [6]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [9]:
num_topics=25

In [10]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-8-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 34.7 s


In [11]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 4min 50s


In [12]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [13]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=num_topics, id2word=common_dictionary)#, passes=10)

Wall time: 51.1 s


In [14]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [15]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(1, 0.04861038), (12, 0.7130223), (22, 0.10017064), (23, 0.11948718)]

In [16]:
x=lda.show_topics(num_topics=num_topics, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: год который проект население технология также область
topic_1: ракета пенсия мышь израиль сосед сша компьютер
topic_2: который год это украина свой украинский страна
topic_3: это мочь который всё человек научный компания
topic_4: антонов нация сербия близость сербский вечеринка серб
topic_5: военный банк который млрд год сша тыс
topic_6: год стать который млн тыс университет nn
topic_7: фестиваль мероприятие эстония латвия литва корь корзина
topic_8: фонд исследование пациент китай китайский изучение кризис
topic_9: рубль год суд млн доллар сумма размер
topic_10: район nn вода участок квартира авария катастрофа
topic_11: это год россия который страна nn российский
topic_12: рак взрыв смерть nn первый физика команда
topic_13: погибнуть миссия советовать майкл провинция взрывной южноафриканский
topic_14: газ запуск медицина испытание установка надёжный стандарт
topic_15: девочка вирус кг трансляция лечить сражение сибирь
topic_16: граница белоруссия активность остров доклад сниз

In [17]:
# text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(num_topics):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [18]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(num_topics)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(num_topics)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.353021,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.013164,0.0,0.0,0.0,0.000000,0.0,0.000000,0.091615,0.0
1,4896,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.055125,0.0,0.0,0.0,0.057946,0.0,0.000000,0.000000,0.0
2,4897,0.000000,0.048632,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.100189,0.119393,0.0
3,4898,0.000000,0.000000,0.0,0.627579,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,4899,0.327769,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.572323,0.0,0.000000,0.000000,0.0


In [19]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [20]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(num_topics)]].values))

In [21]:
doc_dict[293622]

array([0.07196598, 0.        , 0.14339682, 0.        , 0.        ,
       0.        , 0.38167974, 0.        , 0.05593456, 0.05397232,
       0.        , 0.20259112, 0.04344178, 0.        , 0.        ,
       0.        , 0.03670127, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [22]:
results = {
    'method': [],
    'roc_auc': [],
    'precision': [],
    'recall': [],
    'f_score': []
}

### Получение весов-коэффициентов idf

In [23]:
vectorizer = TfidfVectorizer()

weights = vectorizer.fit_transform(users['articles'])

weights_idf = pd.DataFrame(vectorizer.idf_, index=vectorizer.get_feature_names(), columns=['weights'])

### MEAN

In [24]:
def get_user_embedding(user_articles_list, func=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, 0)
    return user_vector

In [25]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [26]:
functions = [np.mean, np.median, np.max]

In [27]:
for function in functions:
    func_name = str(function).split()[1]
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x,function), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(num_topics)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(num_topics)]]
    user_embeddings.head(3)
    
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(num_topics)]], 
                                                    X['churn'], random_state=42)
    
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    
    
    preds = logreg.predict_proba(X_test)[:, 1]
    preds[:10]
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print(func_name)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    print("-"*10)
    results['method'].append(func_name)
    results['roc_auc'].append(roc_auc_score(y_test, preds))
    results['precision'].append(precision[ix])
    results['recall'].append(recall[ix])
    results['f_score'].append(fscore[ix])
    
    
    roc_auc_score(y_test, preds)

mean
Best Threshold=0.247729, F-Score=0.623, Precision=0.566, Recall=0.692
----------
median
Best Threshold=0.271208, F-Score=0.726, Precision=0.663, Recall=0.802
----------
amax
Best Threshold=0.313535, F-Score=0.758, Precision=0.738, Recall=0.779
----------


In [28]:
res = pd.DataFrame(data=results).sort_values('f_score', ascending=False)

In [29]:
res

,method,roc_auc,precision,recall,f_score
2,amax,0.962463,0.737828,0.778656,0.757692
1,median,0.964925,0.663399,0.802372,0.726297
0,mean,0.931247,0.566343,0.691700,0.622776


In [36]:
# res.to_csv('res25.csv', index=False, encoding='utf-8', sep=',')
# res

In [37]:
res10 = pd.read_csv('res10.csv')
res10

,method,roc_auc,precision,recall,f_score
0,amax,0.934607,0.630282,0.707510,0.666667
1,mean,0.896930,0.546763,0.600791,0.572505
2,median,0.880629,0.440299,0.699605,0.540458


In [38]:
res20 = pd.read_csv('res20.csv')
res20

,method,roc_auc,precision,recall,f_score
0,amax,0.966305,0.780000,0.770751,0.775348
1,median,0.971979,0.744526,0.806324,0.774194
2,mean,0.960388,0.677083,0.770751,0.720887


In [39]:
res25 = pd.read_csv('res25.csv')
res25

,method,roc_auc,precision,recall,f_score
0,amax,0.962463,0.737828,0.778656,0.757692
1,median,0.964925,0.663399,0.802372,0.726297
2,mean,0.931247,0.566343,0.691700,0.622776


In [40]:
res30 = pd.read_csv('res30.csv')
res30

,method,roc_auc,precision,recall,f_score
0,median,0.975319,0.751852,0.802372,0.776291
1,amax,0.961669,0.818182,0.675889,0.740260
2,mean,0.945861,0.615385,0.695652,0.653061


Максимальный f_score получается при 30 топиках и использовании np.median